In [5]:
import Pkg; Pkg.activate(joinpath(@__DIR__)); Pkg.instantiate();
using BilinearControl
using BilinearControl.Problems
using BilinearControl.EDMD
import RobotDynamics as RD
using LinearAlgebra
using RobotZoo
using JLD2
using SparseArrays
using Plots
using Distributions
using Distributions: Normal
using Random
using FiniteDiff, ForwardDiff
using Test
using Rotations

  Activating project at `~/.julia/dev/BilinearControl/examples`


In [6]:
Threads.nthreads()

10

In [7]:
# include("learned_models/edmd_utils.jl");

In [8]:
## Visualizer
model = RobotZoo.Quadrotor{MRP{Float64}}()
include(joinpath(Problems.VISDIR, "visualization.jl"))
vis = Visualizer()
delete!(vis)
set_quadrotor!(vis, model)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/jeonghun/.julia/packages/MeshCat/Ax8pH/src/visualizer.jl:73


false

In [9]:
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8700))

# Define Full Quadrotor Model

In [10]:
model = RobotZoo.Quadrotor{MRP{Float64}}()
dmodel = RD.DiscretizedDynamics{RD.RK4}(model)
tf = 5.0
dt = 0.05

T_ref = range(0,tf,step=dt);

In [11]:
n, m = RD.dims(model)
n

12

# Generate Data

In [51]:
Random.seed!(1)

# number of trajectories
num_train = 30
num_test = 20

# Generate a stabilizing LQR controller
Qlqr = Diagonal(fill(1.0, 12))
Rlqr = Diagonal([1e-4, 1e-4, 1e-4, 1e-4])
xe = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ue = [9.81*0.5/4.0, 9.81*0.5/4.0, 9.81*0.5/4.0, 9.81*0.5/4.0]
ctrl_lqr = LQRController(dmodel, Qlqr, Rlqr, xe, ue, dt)

# Sample a bunch of initial conditions for the LQR controller
x0_train_sampler = Product([
    Uniform(-1.0,1.0),
    Uniform(-1.0,1.0),
    Uniform(-1.0,1.0),
    Uniform(-deg2rad(70),deg2rad(70)),
    Uniform(-deg2rad(70),deg2rad(70)),
    Uniform(-deg2rad(70),deg2rad(70)),
    Uniform(-0.5,0.5),
    Uniform(-0.5,0.5),
    Uniform(-0.5,0.5),
    Uniform(-0.25,0.25),
    Uniform(-0.25,0.25),
    Uniform(-0.25,0.25)
])

x0_test_sampler = Product([
    Uniform(-2.0,2.0),
    Uniform(-2.0,2.0),
    Uniform(-2.0,2.0),
    Uniform(-deg2rad(80),deg2rad(80)),
    Uniform(-deg2rad(80),deg2rad(80)),
    Uniform(-deg2rad(80),deg2rad(80)),
    Uniform(-1,1),
    Uniform(-1,1),
    Uniform(-1,1),
    Uniform(-0.25,0.25),
    Uniform(-0.25,0.25),
    Uniform(-0.25,0.25)
])

initial_conditions_train = [rand(x0_train_sampler) for _ in 1:num_train]
initial_conditions_test = [rand(x0_test_sampler) for _ in 1:num_test]

initial_conditions_train = map((x) -> vcat(x[1:3], Rotations.params(MRP(RotXYZ(x[4], x[5], x[6]))), 
    x[7:end]), initial_conditions_train)
initial_conditions_test = map((x) -> vcat(x[1:3], Rotations.params(MRP(RotXYZ(x[4], x[5], x[6]))), 
    x[7:end]), initial_conditions_test)

# Create data set
X_train, U_train = create_data(dmodel, ctrl_lqr, initial_conditions_train, tf, dt)
X_test, U_test = create_data(dmodel, ctrl_lqr, initial_conditions_test, tf*1.5, dt);

In [52]:
@test all(x->x<0.1, map(x->norm(x-xe), X_train[end,:]))
@test all(x->x<0.1, map(x->norm(x-xe), X_test[end,:]))

Test Passed
  Expression: all((x->begin
            x < 0.1
        end), map((x->begin
                norm(x - xe)
            end), X_test[end, :]))

In [24]:
#save data
jldsave(joinpath(Problems.DATADIR, "rex_full_quadrotor_lqr_trajectories.jld2"); 
    X_train, U_train, X_test, U_test, tf, dt)

# Import Training/Test Data

In [ ]:
lqr_traj = load(joinpath(Problems.DATADIR, "rex_full_quadrotor_lqr_trajectories.jld2"))

X_train = lqr_traj["X_train"]
U_train = lqr_traj["U_train"]
X_test = lqr_traj["X_test"]
U_test = lqr_traj["U_test"]
tf = lqr_traj["tf"]
dt = lqr_traj["dt"]

T_ref = range(0,tf,step=dt);

# Fit the Data using Nominal eDMD
This uses the standard eDMD model method used in the CalTech paper, which learns a model of the form:

$$ x^+ = A x + \sum_{i=1}^m u_i C_i x $$

In [ ]:
# Define the basis functions
eigfuns = ["state", "monomial"]
eigorders = [[0],[2, 2]];

In [ ]:
# Fit the data
Z_train, Zu_train, kf = build_eigenfunctions(X_train, U_train, eigfuns, eigorders);

In [ ]:
# Learn Nominal model

# A, B, C, g = learn_bilinear_model(X_train, Z_train, Zu_train,
#     ["ridge", "na"]; 
#     edmd_weights=[1e-6], 
#     mapping_weights=[0.0], 
#     algorithm=:qr
# );

A, B, C, g = learn_bilinear_model(X_train, Z_train, Zu_train,
    ["na", "na"]; 
    edmd_weights=[0.0], 
    mapping_weights=[0.0], 
    algorithm=:qr
);

In [ ]:
# Save model
jldsave(joinpath(Problems.DATADIR,"rex_full_quadrotor_lqr_nominal_eDMD_data.jld2"); A, B, C, g, kf, eigfuns, eigorders, tf, dt)

# Import Nominal EDMD Model

In [ ]:
cartpole_data = load(joinpath(Problems.DATADIR, "rex_full_quadrotor_lqr_nominal_eDMD_data.jld2"))
A_nom = cartpole_data["A"]
B_nom = cartpole_data["B"]
C_nom = cartpole_data["C"]
g = cartpole_data["g"]
kf = cartpole_data["kf"]
tf = cartpole_data["tf"]
dt = cartpole_data["dt"];

# Evaluate the Nominal Fit
The fit is evaluated using:
$$ \frac{1}{P} \sum_{j=1}^P || g (A \varphi(x_j) + \sum_{i=1}^m u_{j,i} C_i \varphi(x_j)) - x_j^+ || $$

where $y = \varphi(x)$ is the Koopman transform.

In [ ]:
err_train = BilinearControl.EDMD.fiterror(A_nom, B_nom, C_nom, g, kf, X_train, U_train)
err_test = BilinearControl.EDMD.fiterror(A_nom, B_nom, C_nom, g, kf, X_test, U_test)
println("Train Error: ", err_train)
println("Test Error:  ", err_test)

### Define the Bilinear Model

In [ ]:
model_bilinear = EDMDModel(A_nom,B_nom,C_nom,g,kf,dt,"full_quadrotor")
dmodel_bilinear = EDMD.ProjectedEDMDModel(model_bilinear)
n,m = RD.dims(model_bilinear)
n0 = EDMD.originalstatedim(model_bilinear)
println("New state dimension: ", n)

### Compare Linearization
To see if the learned model captures the dynamics, we'll evaluate the effective `A` and `B` dynamics Jaocbians of the learned model on the original dynamics, and compare to the nominal Jacobians.

In [ ]:
# Define the equilibrium
xe = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
ue = Problems.trim_controls(model)
ze = RD.KnotPoint{n0,m}(xe,ue,0.0,dt)
ye = EDMD.expandstate(model_bilinear, xe);

In [ ]:
# Nominal Jacobians
J = zeros(n0,n0+m)
xn = zeros(n0)
RD.jacobian!(RD.InPlace(), RD.ForwardAD(), dmodel, J, xn, ze)
A_og = J[:,1:n0]
B_og = J[:,n0+1:end];

In [ ]:
# Bilinear Jacobians
function dynamics_bilinear(x,u,t,dt)
    y = EDMD.expandstate(model_bilinear, x)
    yn = zero(y)
    RD.discrete_dynamics!(model_bilinear, yn, y, u, t, dt)
    EDMD.originalstate(model_bilinear, yn)
end

A_bil_nom = FiniteDiff.finite_difference_jacobian(x->dynamics_bilinear(x,ue,0.0,dt), xe)
B_bil_nom = FiniteDiff.finite_difference_jacobian(u->dynamics_bilinear(xe,u,0.0,dt), ue);

In [ ]:
A_og

In [ ]:
A_bil_nom

In [ ]:
B_og

In [ ]:
B_bil_nom

# Try Stabilizing Nominal Model using LQR

In [ ]:
# Calculate LQR Gain 
Qlqr = Diagonal([10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4])
Rlqr = Diagonal([1e-4, 1e-4, 1e-4, 1e-4])

K_og, = EDMD.dlqr(A_og, B_og, Qlqr, Rlqr)
K_bil_nom, = EDMD.dlqr(A_bil_nom, B_bil_nom, Qlqr, Rlqr);

In [ ]:
# Evaluate stability
isstable_nominal = maximum(abs.(eigvals(A_og - B_og*K_og))) < 1.0
isstable_bilinear = maximum(abs.(eigvals(A_bil_nom - B_bil_nom*K_bil_nom))) < 1.0
isstable_nominal_with_bilinear = maximum(abs.(eigvals(A_og - B_og*K_bil_nom))) < 1.0

println("Stability Summary:")
println("  Dynamics  |  Controller  |  is stable? ")
println("------------|--------------|--------------")
println("  Nominal   |  Nominal     |  ", isstable_nominal)
println("  Bilinear  |  Bilinear    |  ", isstable_bilinear)
println("  Nominal   |  Bilinear    |  ", isstable_nominal_with_bilinear)

In [ ]:
# Simulate nominal model with LQR gain from bilinear model

tf_sim = 5.0
Tsim_lqr_nominal = range(0,tf_sim,step=dt)

x0 = [-0.5, -0.5, 0.5, 0.15, 0.15, 0.16, -1.0, 1.0, 1.0, 0.0, 0.0, 0.0]

ctrl_lqr_og = EDMD.LQRController(dmodel, Qlqr, Rlqr, xe, ue, dt, max_iters=10000, verbose=true)
ctrl_lqr_nominal = EDMD.LQRController(dmodel_bilinear, Qlqr, Rlqr, xe, ue, dt, max_iters=10000, verbose=true)

Xsim_lqr_og, = EDMD.simulatewithcontroller(dmodel, ctrl_lqr_og, x0, tf_sim, dt)
Xsim_lqr_nominal, = EDMD.simulatewithcontroller(dmodel, ctrl_lqr_nominal, x0, tf_sim, dt)

plotstates(Tsim_lqr_nominal, Xsim_lqr_og, inds=1:3, xlabel="time (s)", ylabel="states",
            label=["x (og dynamics)" "y (og dynamics)" "z (og dynamics)"], legend=:topright, lw=2,
            linestyle=:dash, color=[1 2 3])
plotstates!(Tsim_lqr_nominal, Xsim_lqr_nominal, inds=1:3, xlabel="time (s)", ylabel="states",
            label=["x (nominal eDMD)" "y (nominal eDMD)" "z (nominal eDMD)"], legend=:topright, lw=2,
            color=[1 2 3])
ylims!((-0.75, 0.75))

In [ ]:
plotstates(Tsim_lqr_nominal, Xsim_lqr_og, inds=4:6, xlabel="time (s)", ylabel="states",
            label=["MRP-x (og dynamics)" "MRP-y (og dynamics)" "MRP-z (og dynamics)"], legend=:topright, lw=2,
            linestyle=:dash, color=[1 2 3])
plotstates!(Tsim_lqr_nominal, Xsim_lqr_nominal, inds=4:6, xlabel="time (s)", ylabel="states",
            label=["MRP-x (nominal eDMD)" "MRP-y (nominal eDMD)" "MRP-z (nominal eDMD)"], legend=:topright, lw=2,
            color=[1 2 3])

# Evaluate New eDMD Model with Penalties on Jacobians
Here we incorporate derivative data by penalizing the difference of the Jacobian of the learned model from the expected Jacobians of the nominal model, e.g.:

$$ \sum_{j=1}^P || \frac{\partial}{\partial x} G \hat{f}(\varphi(x_j), u_j) - A_j ||^2 $$

where 

$$ \hat{f}(y,u) = A y + B u + \sum_{i=1}^m u_i C_i y $$

are the learned bilinear dynamics.

In [ ]:
# Generate Jacobians
xn = zeros(n0)
jacobians = map(CartesianIndices(U_train)) do cind
    k = cind[1]
    x = X_train[cind]
    u = U_train[cind]
    z = RD.KnotPoint{n0,m}(x,u,T_ref[k],dt)
    J = zeros(n0,n0+m)
    RD.jacobian!(
        RD.InPlace(), RD.ForwardAD(), dmodel, J, xn, z 
    )
    J
end
A_train = map(J->J[:,1:n0], jacobians)
B_train = map(J->J[:,n0+1:end], jacobians)

# Convert states to lifted Koopman states
Y_train = map(kf, X_train)

# Calculate Jacobian of Koopman transform
F_train = map(@view X_train[1:end-1,:]) do x
    sparse(ForwardDiff.jacobian(x->EDMD.expandstate(model_bilinear,x), x))
end;

### Build Least-squares problem
Find the eDMD data `A`, `B`, and `C` that minimize the following residual:

$$\sum_{j=1}^P (1-\alpha)|| A y_j + B u_j + \sum_{i=1}^m u_{j,i} C_i x_j - y_j^+ ||^2 + 
\alpha || G (A + \sum_{i=1}^m u_{j,i} C_j) F_j  - A_j ||^2 + 
\alpha || G (B + [C_1 x_j \dots C_m x_j]) - B_j ||^2  $$

where $F_j = \partial \varphi(x_j) / \partial x$.

This is equivalent to 

$$ (1-\alpha)|| E Z_{1:P} - Y_{1:P}^+ ||^2 + 
\alpha || G E \hat{A}_{1:P} - A_{1:P} ||^2 +
\alpha || G E \hat{B}_{1:P} - B_{1:P} ||^2 $$

where

$$ E = \begin{bmatrix} A & B & C_1 & \dots & C_m \end{bmatrix} \in \mathbb{R}^{n \times p} $$
$$ Z_j = \begin{bmatrix} 
    x_j \\ u_j \\ u_{j,1} x_j \\ \vdots \\ u_{j,m} x_j 
\end{bmatrix} \in \mathbb{R}^{p}, \quad
\hat{A}_j = \begin{bmatrix} 
    I_n \\ 0_{m,n} \\ u_{j,1} I_n \\ \vdots \\ u_{j,m} I_n 
\end{bmatrix} \in \mathbb{R}^{p \times n}, \quad
\hat{B}_j = \begin{bmatrix} 
    0_{n,m} \\ I_m \\ [x_j \; 0 \; \dots \; 0] \\ \vdots \\ [0 \; \dots \; 0 \; x_j] 
\end{bmatrix} \in \mathbb{R}^{p \times m}
$$

which is equivalent to 

$$ \left\lVert 
\begin{bmatrix} 
    Z_{1:P}^T \otimes I_n \\
    \hat{A}_{1:P}^T \otimes G \\
    \hat{B}_{1:P}^T \otimes G 
\end{bmatrix} \text{vec}(E) - 
\begin{bmatrix}
    \text{vec}{Y_{1:P}^+}\\
    \text{vec}{A_{1:P}^+}\\
    \text{vec}{B_{1:P}^+}\\
\end{bmatrix}
\right\rVert^2 $$

In [ ]:
# Create a sparse version of the G Jacobian
G = spdiagm(n0,n,1=>ones(n0)) 
@test norm(G - model_bilinear.g) < 1e-8

# Build Least Squares Problem
W,s = BilinearControl.EDMD.build_edmd_data(
    Z_train, U_train, A_train, B_train, F_train, model_bilinear.g
)

n = length(Z_train[1]);

### Solve the Least-squares Problem

In [ ]:
@time Wsparse = sparse(W);

In [ ]:
@show size(Wsparse);

In [ ]:
BilinearControl.matdensity(Wsparse)

In [ ]:
# Solve directly
 
# @time F = qr(Wsparse)
# @time x = F \ s
# norm(W*x - s)
# E = reshape(x,n,:)

In [ ]:
# Solve with RLS

@time x_rls = BilinearControl.EDMD.rls_qr(Vector(s), Wsparse; Q=1e-6)
E = reshape(x_rls,n,:);

In [ ]:
# Extract out bilinear dynamics
A = E[:,1:n]
B = E[:,n .+ (1:m)]
C = E[:,n+m .+ (1:n*m)]

C_list = Matrix{Float64}[]
    
for i in 1:m
    C_i = C[:, (i-1)*n+1:i*n]
    push!(C_list, C_i)
end

return A, B, C_list, Matrix(g)

C = C_list;

In [ ]:
# Save data
jldsave(joinpath(Problems.DATADIR,"rex_full_quadrotor_lqr_jacobian_penalized_eDMD_data.jld2"); A, B, C, g, kf, eigfuns, eigorders, tf, dt)

# Import New Model

In [ ]:
cartpole_data = load(joinpath(Problems.DATADIR, "rex_full_quadrotor_lqr_jacobian_penalized_eDMD_data.jld2"))
A_jacpen = cartpole_data["A"]
B_jacpen = cartpole_data["B"]
C_jacpen = cartpole_data["C"]
g = cartpole_data["g"]
kf = cartpole_data["kf"]
tf = cartpole_data["tf"]
dt = cartpole_data["dt"]

T_ref = range(0,tf,step=dt);

# Evaluate the New Fit

In [ ]:
# Evaluate the Fit
err_train2 = BilinearControl.EDMD.fiterror(A_jacpen, B_jacpen, C_jacpen, g, kf, X_train, U_train)
err_test2 = BilinearControl.EDMD.fiterror(A_jacpen, B_jacpen, C_jacpen, g, kf, X_test, U_test)
println("Train Error: ", err_train)
println("Test Error:  ", err_test)
println("")
println("New Train Error: ", err_train2)
println("New Test Error:  ", err_test2)

In [ ]:
model_bilinear_jacpen = EDMDModel(A_jacpen, B_jacpen, C_jacpen, g, kf, dt, "rex_full_quadrotor")
dmodel_bilinear_jacpen = EDMD.ProjectedEDMDModel(model_bilinear_jacpen)
n,m = RD.dims(model_bilinear_jacpen)
n0 = EDMD.originalstatedim(model_bilinear_jacpen);

In [ ]:
# Get A,B for new system
function dynamics_bilinear_jacpen(x,u,t,dt)
    y = EDMD.expandstate(model_bilinear_jacpen, x)
    yn = zero(y)
    RD.discrete_dynamics!(model_bilinear_jacpen, yn, y, u, t, dt)
    EDMD.originalstate(model_bilinear_jacpen, yn)
end

A_bil_jacpen = FiniteDiff.finite_difference_jacobian(x->dynamics_bilinear_jacpen(x,ue,0.0,dt), xe)
B_bil_jacpen = FiniteDiff.finite_difference_jacobian(u->dynamics_bilinear_jacpen(xe,u,0.0,dt), ue);

**Compare the Jacobians with Nominal model**

In [ ]:
A_og

In [ ]:
A_bil_jacpen

In [ ]:
B_og

In [ ]:
B_bil_jacpen

# Try Stabilizing New Model using LQR

In [ ]:
K_bil_jacpen, = EDMD.dlqr(A_bil_jacpen, B_bil_jacpen, Qlqr, Rlqr);

In [ ]:
# Evaluate stability
isstable_bilinear2 = maximum(abs.(eigvals(A_bil_jacpen - B_bil_jacpen*K_bil_jacpen))) < 1.0
isstable_nominal_with_bilinear2 = maximum(abs.(eigvals(A_og - B_og*K_bil_jacpen))) < 1.0

println("Stability Summary:")
println("  Dynamics  |  Controller  |  is stable? ")
println("------------|--------------|--------------")
println("  Nominal   |  Nominal     |  ", isstable_nominal)
println("  Bilinear  |  Bilinear    |  ", isstable_bilinear2)
println("  Nominal   |  Bilinear    |  ", isstable_nominal_with_bilinear2)

In [ ]:
tf_sim = 5.0
Tsim_lqr_jacpen = range(0,tf_sim,step=dt)

x0 = [-0.5, -0.5, 0.5, 0.15, 0.15, 0.16, -1.0, 1.0, 1.0, 0.0, 0.0, 0.0]

ctrl_lqr_jacpen = EDMD.LQRController(dmodel_bilinear_jacpen, Qlqr, Rlqr, xe, ue, dt, max_iters=10000, verbose=true)
Xsim_lqr_jacpen, = EDMD.simulatewithcontroller(dmodel, ctrl_lqr_jacpen, x0, tf_sim, dt)

plotstates(Tsim_lqr_nominal, Xsim_lqr_og, inds=1:3, xlabel="time (s)", ylabel="states",
            label=["x (og dynamics)" "y (og dynamics)" "z (og dynamics)"], legend=:topright, lw=2,
            linestyle=:dash, color=[1 2 3])
plotstates!(Tsim_lqr_nominal, Xsim_lqr_nominal, inds=1:3, xlabel="time (s)", ylabel="states",
            label=["x (nominal eDMD)" "y (nominal eDMD)" "z (nominal eDMD)"], legend=:topright, lw=2,
            linestyle=:dot, color=[1 2 3])
plotstates!(Tsim_lqr_jacpen, Xsim_lqr_jacpen, inds=1:3, xlabel="time (s)", ylabel="states",
            label=["x (jacobian eDMD)" "y (jacobian eDMD)" "z (jacobian eDMD)"], legend=:topright, lw=2,
            color=[1 2 3])
ylims!((-0.75, 0.75))

In [ ]:
plotstates(Tsim_lqr_nominal, Xsim_lqr_og, inds=4:6, xlabel="time (s)", ylabel="states",
            label=["MRP-x (og dynamics)" "MRP-y (og dynamics)" "MRP-z (og dynamics)"], legend=:topright, lw=2,
            linestyle=:dash, color=[1 2 3])
plotstates!(Tsim_lqr_nominal, Xsim_lqr_nominal, inds=4:6, xlabel="time (s)", ylabel="states",
            label=["MRP-x (nominal eDMD)" "MRP-y (nominal eDMD)" "MRP-z (nominal eDMD)"], legend=:topright, lw=2,
            linestyle=:dot, color=[1 2 3])
plotstates!(Tsim_lqr_jacpen, Xsim_lqr_jacpen, inds=4:6, xlabel="time (s)", ylabel="states",
            label=["MRP-x (jacobian eDMD)" "MRP-y (jacobian eDMD)" "MRP-z (jacobian eDMD)"], legend=:topright, lw=2,
            color=[1 2 3])
ylims!((-0.15, 0.25))

In [ ]:
render(vis)

In [ ]:
# visualize!(vis, model, tf_sim, Xsim_lqr_nominal)
visualize!(vis, model, tf_sim, Xsim_lqr_jacpen)